Deep Learning
=============

Assignment 2
------------

Previously in `1_notmnist.ipynb`, we created a pickle with formatted datasets for training, development and testing on the [notMNIST dataset](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html).

The goal of this assignment is to progressively train deeper and more accurate models using TensorFlow.

In [2]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range

First reload the data we generated in `1_notmnist.ipynb`.

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [3]:
d = np.random.rand(10, 3, 3)
l = np.random.rand(10).astype(np.int32)

d = d.reshape(-1, 5).astype(np.float32)
d

array([[ 0.10478416,  0.88294101,  0.65875107,  0.2853514 ,  0.90840465],
       [ 0.59943146,  0.72686136,  0.43134499,  0.98380655,  0.78594732],
       [ 0.77226454,  0.04625027,  0.22800039,  0.38043603,  0.76551253],
       [ 0.16965829,  0.3227241 ,  0.58612311,  0.65878713,  0.97928554],
       [ 0.70420676,  0.13954751,  0.95513278,  0.00587442,  0.42358449],
       [ 0.33917919,  0.90692163,  0.48066294,  0.86730105,  0.79570532],
       [ 0.08848242,  0.1386705 ,  0.99068111,  0.75900888,  0.76483792],
       [ 0.8537755 ,  0.98619974,  0.91815102,  0.11630385,  0.28584409],
       [ 0.27769658,  0.17825252,  0.11504264,  0.46982616,  0.80807328],
       [ 0.46180728,  0.63762158,  0.46523991,  0.3763653 ,  0.0263969 ],
       [ 0.23595342,  0.59613878,  0.2856985 ,  0.23036787,  0.49679542],
       [ 0.32043558,  0.70216352,  0.41501376,  0.0750792 ,  0.83073819],
       [ 0.33564973,  0.21994624,  0.34052452,  0.52552235,  0.17858617],
       [ 0.4359898 ,  0.89309174,  0.0

In [4]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 0 to [1.0, 0.0, 0.0 ...], 1 to [0.0, 1.0, 0.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


We're first going to train a multinomial logistic regression using simple gradient descent.

TensorFlow works like this:
* First you describe the computation that you want to see performed: what the inputs, the variables, and the operations look like. These get created as nodes over a computation graph. This description is all contained within the block below:

      with graph.as_default():
          ...

* Then you can run the operations on this graph as many times as you want by calling `session.run()`, providing it outputs to fetch from the graph that get returned. This runtime operation is all contained in the block below:

      with tf.Session(graph=graph) as session:
          ...

Let's load all the data into TensorFlow and build the computation graph corresponding to our training:

In [5]:
[image_size * image_size, num_labels]

[784, 10]

In [6]:
# With gradient descent training, even this much data is prohibitive.
# Subset the training data for faster turnaround.
train_subset = 10000

graph = tf.Graph()
with graph.as_default():

  # Input data.
  # Load the training, validation and test data into constants that are
  # attached to the graph.
  tf_train_dataset = tf.constant(train_dataset[:train_subset, :])
  tf_train_labels = tf.constant(train_labels[:train_subset])
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  # These are the parameters that we are going to be training. The weight
  # matrix will be initialized using random valued following a (truncated)
  # normal distribution. The biases get initialized to zero.
  weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  # We multiply the inputs with the weight matrix, and add biases. We compute
  # the softmax and cross-entropy (it's one operation in TensorFlow, because
  # it's very common, and it can be optimized). We take the average of this
  # cross-entropy across all training examples: that's our loss.
  logits = tf.matmul(tf_train_dataset, weights) + biases
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
  
  # Optimizer.
  # We are going to find the minimum of this loss using gradient descent.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  # These are not part of training, but merely here so that we can report
  # accuracy figures as we train.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf_valid_dataset, weights) + biases)
  test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

Let's run this computation and iterate:

In [7]:
num_steps = 801

def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

with tf.Session(graph=graph) as session:
  # This is a one-time operation which ensures the parameters get initialized as
  # we described in the graph: random weights for the matrix, zeros for the
  # biases. 
  tf.initialize_all_variables().run()
  print('Initialized')
  for step in range(num_steps):
    # Run the computations. We tell .run() that we want to run the optimizer,
    # and get the loss value and the training predictions returned as numpy
    # arrays.
    _, l, predictions = session.run([optimizer, loss, train_prediction])
    if (step % 100 == 0):
      print('Loss at step %d: %f' % (step, l))
      print('Training accuracy: %.1f%%' % accuracy(
        predictions, train_labels[:train_subset, :]))
      # Calling .eval() on valid_prediction is basically like calling run(), but
      # just to get that one numpy array. Note that it recomputes all its graph
      # dependencies.
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Loss at step 0: 16.694256
Training accuracy: 8.3%
Validation accuracy: 11.9%
Loss at step 100: 2.324809
Training accuracy: 71.6%
Validation accuracy: 70.3%
Loss at step 200: 1.844987
Training accuracy: 74.8%
Validation accuracy: 73.1%
Loss at step 300: 1.593498
Training accuracy: 76.5%
Validation accuracy: 74.2%
Loss at step 400: 1.426128
Training accuracy: 77.3%
Validation accuracy: 74.6%
Loss at step 500: 1.303646
Training accuracy: 78.1%
Validation accuracy: 74.7%
Loss at step 600: 1.208654
Training accuracy: 78.7%
Validation accuracy: 74.9%
Loss at step 700: 1.132072
Training accuracy: 78.9%
Validation accuracy: 75.0%
Loss at step 800: 1.068589
Training accuracy: 79.2%
Validation accuracy: 75.1%
Test accuracy: 83.2%


Let's now switch to stochastic gradient descent training instead, which is much faster.

The graph will be similar, except that instead of holding all the training data into a constant node, we create a `Placeholder` node which will be fed actual data at every call of `session.run()`.

In [8]:
batch_size = 128

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  logits = tf.matmul(tf_train_dataset, weights) + biases
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf_valid_dataset, weights) + biases)
  test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

Let's run it:

In [9]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 16.169407
Minibatch accuracy: 10.2%
Validation accuracy: 14.9%
Minibatch loss at step 500: 1.760832
Minibatch accuracy: 73.4%
Validation accuracy: 75.3%
Minibatch loss at step 1000: 1.650412
Minibatch accuracy: 71.1%
Validation accuracy: 77.0%
Minibatch loss at step 1500: 0.736809
Minibatch accuracy: 84.4%
Validation accuracy: 77.2%
Minibatch loss at step 2000: 0.955679
Minibatch accuracy: 78.9%
Validation accuracy: 77.5%
Minibatch loss at step 2500: 0.930589
Minibatch accuracy: 78.9%
Validation accuracy: 78.3%
Minibatch loss at step 3000: 1.300719
Minibatch accuracy: 75.8%
Validation accuracy: 78.8%
Test accuracy: 86.0%


---
Problem
-------

Turn the logistic regression example with SGD into a 1-hidden layer neural network with rectified linear units [nn.relu()](https://www.tensorflow.org/versions/r0.7/api_docs/python/nn.html#relu) and 1024 hidden nodes. This model should improve your validation / test accuracy.

---

In [14]:
batch_size = 128
num_hidden_nodes = 1024

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights_01 = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_hidden_nodes]))
  biases_01 = tf.Variable(tf.zeros([num_hidden_nodes]))
    
  weights_12 = tf.Variable(tf.truncated_normal([num_hidden_nodes, num_labels]))
  biases_12 = tf.Variable(tf.zeros([num_labels]))

  # Training computation.
  z_01 = tf.matmul(tf_train_dataset, weights_01) + biases_01
  h_1 = tf.nn.relu(z_01)
  z_12 = tf.matmul(h_1, weights_12) + biases_12
  
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(z_12, tf_train_labels))
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(z_12)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf.nn.relu(tf.matmul(tf_valid_dataset, weights_01) + biases_01), weights_12) + biases_12)
  test_prediction = tf.nn.softmax(
    tf.matmul(tf.nn.relu(tf.matmul(tf_test_dataset, weights_01) + biases_01), weights_12) + biases_12)

In [16]:
num_steps = 30001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 353.645355
Minibatch accuracy: 4.7%
Validation accuracy: 31.4%
Minibatch loss at step 500: 28.377028
Minibatch accuracy: 81.2%
Validation accuracy: 77.9%
Minibatch loss at step 1000: 10.635538
Minibatch accuracy: 77.3%
Validation accuracy: 78.4%
Minibatch loss at step 1500: 3.168049
Minibatch accuracy: 83.6%
Validation accuracy: 81.2%
Minibatch loss at step 2000: 8.806160
Minibatch accuracy: 82.8%
Validation accuracy: 81.0%
Minibatch loss at step 2500: 2.551536
Minibatch accuracy: 82.8%
Validation accuracy: 81.9%
Minibatch loss at step 3000: 9.843122
Minibatch accuracy: 80.5%
Validation accuracy: 80.0%
Minibatch loss at step 3500: 3.895168
Minibatch accuracy: 78.1%
Validation accuracy: 81.2%
Minibatch loss at step 4000: 3.922864
Minibatch accuracy: 82.0%
Validation accuracy: 81.7%
Minibatch loss at step 4500: 1.668216
Minibatch accuracy: 78.1%
Validation accuracy: 82.3%
Minibatch loss at step 5000: 1.745860
Minibatch accuracy: 84.4%
Validation accu